In [1]:
#import packages
from os import listdir, makedirs
from os.path import isdir
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode        # pypeline engine

from nipype.interfaces.fsl.model import GLM, MELODIC
from pandas import DataFrame, Series

#set output file type for FSL to NIFTI
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
setup='Cat'
sample='newborn' #6mo or newborn

if setup=='Marissa':
    studyhome = '/oak/stanford/groups/iang/BABIES_data/BABIES_rest'
    output_dir = studyhome + '/ica'
    preproc_dir = studyhome + '/preproc'
    workflow_dir = studyhome + '/workflows' 
    templates = '/Volumes/iang/BABIES_data/BABIES_rest/templates'
elif setup=='Cat':
    studyhome = '/home/camachocm2/Analysis/SEA/rest_proc'
    output_dir = studyhome + '/ica'
    preproc_dir = studyhome + '/preproc'
    workflow_dir = studyhome + '/workflows'
    templates = '/home/camachocm2/Analysis/SEA/templates'

if sample =='6mo':
    template_brain = templates + '/6mo_T2w_template_2mm.nii.gz'
    template_wm = templates + '/6mo_T2w_template_2mm_wm.nii.gz'
    template_mask = templates + '/6mo_T2w_template_2mm_mask.nii.gz'
elif sample=='newborn':
    template_brain = templates + '/neonate_T2w_template_2mm.nii.gz'
    template_wm = templates + '/neonate_T2w_template_2mm_wm.nii.gz'
    template_mask = templates + '/neonate_T2w_template_2mm_gm.nii.gz'


#subjects_list = open(studyhome + '/misc/all_subjects.txt').read().splitlines()
subjects_list = ['N1011']


In [2]:
# Resting state preprocessing
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name='infosource')
infosource.iterables = ('subject_id', subjects_list)

# Datasink- where our select outputs will go
substitutions = [('_subject_id_', '')]
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions


# Data grabber- select fMRI and sMRI
templates = {'func': preproc_dir + '/proc_func/{subject_id}/func_filtered_smooth.nii.gz'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

ica = Node(MELODIC(approach='tica',num_ICs=25, no_bet=False, report=True, mask=template_mask), 
           name='ica')

In [3]:
rs_ica = Workflow(name='rs_ica')
rs_ica.connect([(infosource,selectfiles,[('subject_id','subject_id')]),
                (selectfiles,ica,[('func','in_files')]),
                (ica, datasink, [('out_dir','rest_ica_out')])
               ])

rs_ica.base_dir = workflow_dir
rs_ica.write_graph(graph2use='flat')
rs_ica.run('MultiProc', plugin_args={'n_procs': 2})

190406-17:55:48,825 workflow INFO:
	 Generated workflow graph: /home/camachocm2/Analysis/SEA/rest_proc/workflows/rs_ica/graph.png (graph2use=flat, simple_form=True).
190406-17:55:48,857 workflow INFO:
	 Workflow rs_ica settings: ['check', 'execution', 'logging', 'monitoring']
190406-17:55:48,875 workflow INFO:
	 Running in parallel.
190406-17:55:48,879 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 56.55/56.55, Free processors: 2/2.
190406-17:55:48,960 workflow INFO:
	 [Node] Setting-up "rs_ica.selectfiles" in "/home/camachocm2/Analysis/SEA/rest_proc/workflows/rs_ica/_subject_id_N1011/selectfiles".
190406-17:55:48,969 workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
190406-17:55:48,978 workflow INFO:
	 [Node] Finished "rs_ica.selectfiles".
190406-17:55:50,881 workflow INFO:
	 [Job 0] Completed (rs_ica.selectfiles).
190406-17:55:50,886 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 56